# Student Loan Risk with Deep Learning

In [67]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import numpy as np

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [68]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [69]:
# lets look at more info regarding the dataframe
display(f"shape of loans df is: {loans_df.shape}")
print(f"info for loans df...\n{loans_df.info()}")
# there are no nan to deal with. No imputing needed.
# all datatypes are ok as well.

'shape of loans df is: (1599, 12)'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   payment_history         1599 non-null   float64
 1   location_parameter      1599 non-null   float64
 2   stem_degree_score       1599 non-null   float64
 3   gpa_ranking             1599 non-null   float64
 4   alumni_success          1599 non-null   float64
 5   study_major_code        1599 non-null   float64
 6   time_to_completion      1599 non-null   float64
 7   finance_workshop_score  1599 non-null   float64
 8   cohort_ranking          1599 non-null   float64
 9   total_loan_score        1599 non-null   float64
 10  financial_aid_score     1599 non-null   float64
 11  credit_ranking          1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB
info for loans df...
None


In [70]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [71]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()
# looks pretty balanced.  We'll add in balanced accuracy or classification report at the end to ensure good performance 

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [72]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"].values

# Display a sample of y
print(f"Sanity check: The type of y is: {type(y)}")
y[:5]

Sanity check: The type of y is: <class 'numpy.ndarray'>


array([0, 0, 0, 1, 0], dtype=int64)

In [73]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns="credit_ranking")

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [74]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1199, 11) (400, 11) (1199,) (400,)


### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [75]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)
# Scale both the train and test dataset s based on the fitted scaler
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [76]:
# Define the the number of inputs (features) to the model
num_inputs = len(X_train.columns)

# Review the number of features
display(f"The number of inputs and the number of features is: {num_inputs}")


'The number of inputs and the number of features is: 11'

In [77]:

# Define the number of hidden nodes for the second hidden layer
hidden_nodes = 6
# Define the number of neurons in the output layer
output_nodes = 1

In [78]:
# Create the Sequential model instance
model = Sequential()
# add first visible layer, where there will be one neuron for each feature
model.add(Dense(num_inputs, activation='relu', input_dim=num_inputs))
# Add the hidden layer
model.add(Dense(hidden_nodes, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(output_nodes, activation='sigmoid'))

In [79]:
# Display the Sequential model summary

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 11)                132       
                                                                 
 dense_11 (Dense)            (None, 6)                 72        
                                                                 
 dense_12 (Dense)            (None, 1)                 7         
                                                                 
Total params: 211 (844.00 Byte)
Trainable params: 211 (844.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [80]:
# This is from the starter code. I am leaving this here to comment...

# Display the Sequential model summary
# NOTE: Since there are 11 inputs, I did 11 nodes in my input layer. Having only 6 per the answer key below seems problematic.

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [81]:
# Compile the Sequential model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [82]:
# Fit the model using 50 epochs and the training data
fit_model = model.fit(X_train_scaled, y_train, epochs=50, verbose=1)

Epoch 1/50
38/38 [==============================] - 0s 1ms/step - loss: 0.6877 - accuracy: 0.5588
Epoch 2/50
38/38 [==============================] - 0s 1ms/step - loss: 0.6517 - accuracy: 0.6289
Epoch 3/50
 1/38 [..............................] - ETA: 0s - loss: 0.6228 - accuracy: 0.7188

38/38 [==============================] - 0s 1ms/step - loss: 0.6255 - accuracy: 0.6564
Epoch 4/50
38/38 [==============================] - 0s 888us/step - loss: 0.6048 - accuracy: 0.6764
Epoch 5/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5859 - accuracy: 0.6922
Epoch 6/50
38/38 [==============================] - 0s 1ms/step - loss: 0.5703 - accuracy: 0.7006
Epoch 7/50
38/38 [==============================] - 0s 1ms/step - loss: 0.5588 - accuracy: 0.7131
Epoch 8/50
38/38 [==============================] - 0s 1ms/step - loss: 0.5491 - accuracy: 0.7189
Epoch 9/50
38/38 [==============================] - 0s 1ms/step - loss: 0.5416 - accuracy: 0.7273
Epoch 10/50
38/38 [==============================] - 0s 1ms/step - loss: 0.5349 - accuracy: 0.7298
Epoch 11/50
38/38 [==============================] - 0s 1ms/step - loss: 0.5294 - accuracy: 0.7364
Epoch 12/50
38/38 [==============================] - 0s 1ms/step - loss: 0.5251 - accuracy: 0.7348
Epoch 13/50
38/38 [=======

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [83]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuaracy = model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss %: {round(model_loss*100,2)}, Accuarcy %: {round(model_accuaracy*100,2)}")

13/13 - 0s - loss: 0.5132 - accuracy: 0.7650 - 88ms/epoch - 7ms/step
Loss %: 51.32, Accuarcy %: 76.5


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [47]:
# Set the model's file path
path_and_name = "saved_model/student_loans.keras"

# Export your model to a keras file
model.save(path_and_name)


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [34]:
# Set the model's file path
# this was set above: path_and_name = "saved_model/student_loans.keras"

# Load the model to a new object
loaded_model = load_model(path_and_name)

In [36]:
# inserted this cell as a cross check on what was saved
loaded_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 11)                132       
                                                                 
 dense_5 (Dense)             (None, 6)                 72        
                                                                 
 dense_6 (Dense)             (None, 1)                 7         
                                                                 
Total params: 211 (844.00 Byte)
Trainable params: 211 (844.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [38]:
# Make predictions with the test data
Y_hat = model.predict(X_test_scaled)

# Display a sample of the predictions
Y_hat[:6]

13/13 [==============================] - 0s 848us/step


array([[0.42823917],
       [0.37419438],
       [0.93051267],
       [0.6830534 ],
       [0.9773593 ],
       [0.84122497]], dtype=float32)

In [43]:
# Save the predictions to a DataFrame and round the predictions to binary results
y_hat_rounded = np.round(Y_hat,0)
predict_df = pd.DataFrame(y_hat_rounded,columns=["predictions"])
display(predict_df['predictions'].value_counts())
predict_df

predictions
1.0    202
0.0    198
Name: count, dtype: int64

,predictions
0,0.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,0.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [44]:
# Print the classification report with the y test data and predictions
class_report = classification_report(y_test, predict_df['predictions'])
print("\nClassification Report:\n", class_report)


Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.75      0.73       188
           1       0.77      0.73      0.75       212

    accuracy                           0.74       400
   macro avg       0.74      0.74      0.74       400
weighted avg       0.74      0.74      0.74       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

*Preface: In the Python assignment above, we looked at whether a student would repay a loan largely from the standpoint of someone considering issuing loans. This is only peripherally related to the questions bein this section. I will interpret these questions as referring to a recomendation system for students, and not loan companies or government agencies. The questions seem to clearly be targeted toward the perspective of the student and not toward the lender.*

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

Several types of information would be required to build a recommendation system for students seeking loans.  The categories are financial information (including financial need), demographic information, and educational information. Financial information would include income, their dependent status (i.e. are they listed as someone's, such as their parents) dependents, their employment status, employment plans during their education (are they currently or planning to work part time, full time or not at all), their existing assets versus depts and credit history/ credit scores.  
The financial need would need to be determined. This would include the type of schooling they will be attending, whether they plan to live on-campus or off or do they plan on living with their parents or others that will cover their housing expense. 
Basic demographic information would be needed as well. This includes citizenship, state residency and other factors. Both citizenship and state residency have major impacts on tuition costs. 
Each of these factors would need to be compared against eligibility requirements for government (such as federal student loans) versus private loans, since the interest rates and terms can vary widely. 
Frankly, we would want to also collect the information related to receiving grants, such as PEL grants in case the students could obtain grants to lower loan amounts, etc.
We would need to establish what is the basis of the recommendation. Eligibility and overall repayment rates would be two key criterions. Data on other students’ satisfaction with their loans might be tough to obtain and might not be the best metric. Students with high paying jobs may not be burdened by loan repayment, whereas students who’s jobs with lower incomes might not be so satisfied. 

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

I should note at the onset, that the primary application would utilize a set of business logic (like a flow chart) where information is obtained to determine eligibility to the most favorable loans in terms of rates, payment flexibility, etc. I would see the recommendation system (based on one of the filtering techniques) as an adjunct to the business logic application.
I would lean to using a content-based filtering recommendation system. Prior to discussing why I would recommend content-based filtering, I’ll comment on collaborative and context-based filtering. Collaborative filtering assumes that what people have liked in the past (training data) will continue to be liked in present or future data. However, since loan rates and eligibility requirements can change, I would be concerned about this assumption. Collaborative filtering also has the “cold start” problem. Context-based filtering typically looks at the context, which as the time and place of use/application. This would add complexity, but I’m not sure it would add much additional value.  Content based filtering might be the most appropriate, since it considers the content of the items, rather than just if something was used or liked. Content could include factors related to loan eligibility, financial need, school and living expenses.  

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

1)	One challenge would be to keep the system up to date with respect to loan rates, terms, conditions and eligibility requirements. Just because students in the past have used specific programs does not mean that under the current terms that prospective student would be eligible for the loan or that it would have the best terms.  This is relevant since the goal would be to ensure that the student is getting the best possible terms and conditions as a primary goal of the system.
2)	Beyond published terms and conditions, it would likely be hard to get complete data across both government based versus private sector loans. With private loans, it would be difficult it get consistent data across the different companies offering loans. This is relevant since the system cannot be built without consistent and plentiful data.

